In [3]:
import tensorflow as tf 
import tensorflow_probability as tfp
from tensorflow.keras.datasets import mnist

In [4]:
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

import argparse
import random
import os
import math


tfd = tfp.distributions

In [6]:
import importlib
# import new_custom_layers
import custom_layers
import utilities

In [7]:
# Hyperparameter setup
learning_rate = 0.001
epochs = 100
batch_size = 128
num_classes = 10
num_samples = 1

### MNIST data (5000 train, 5000 val and 10000 test) with 50% noise in training data

In [8]:
# # Load and preprocess MNIST data
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.reshape([-1, 28*28])
# x_test = x_test.reshape([-1, 28*28])
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

# y_train = y_train.astype('int32')
# y_test = y_test.astype('int32')

# train_samples = x_train.shape[0]
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# x_val, y_val = x_test, y_test
n_samples = 5000
I = np.eye(num_classes)
x_train_S, y_train_S, x_test, y_test = utilities.load_mnist(n_samples, square=False, conv=False)
y_train_S = y_train_S.astype('int32')
y_test = y_test.astype('int32')

N = int(n_samples/2)

x_val = x_train_S[N:]
y_val = y_train_S[N:]

x_train = x_train_S[:N]
y_train = y_train_S[:N]

# Add output noise to important class
R = int(.5*N)
r_ind = random.sample(range(0,N),R)
r_class = np.random.randint(0,num_classes,R)
y_train[r_ind] = r_class

train_samples = N

x_train shape: (5000, 784)
5000 train samples
10000 test samples


### Loss matrix 

In [9]:
digits = [3,8]
np.set_printoptions(precision=3)
L = np.copy(I)
L[np.where(I==0)] = 1
L[[3,8]] = 0.7 # Select more important rows with lower loss in prediction
L[np.where(I==1)] = 0

loss_mat = L
M = 1.0001

print("         TRUTH\n" )
string = 'PRED'
for i in range(num_classes):
    if i < 4:
        print(string[i],'  ',str(i),': ',M - loss_mat[i])
    else:
        print('    ',str(i),': ',M - loss_mat[i])
        
# Set up weight for weighted cross entropy

class_weight = np.ones((num_classes))
class_weight[digits] = 1.4

print(loss_mat)

         TRUTH

P    0 :  [1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
R    1 :  [1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
E    2 :  [1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
D    3 :  [0.3 0.3 0.3 1.  0.3 0.3 0.3 0.3 0.3 0.3]
     4 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
     5 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04]
     6 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04]
     7 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04]
     8 :  [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 1.  0.3]
     9 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00]
[[0.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  0.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  0.  1.  1.  1.  1.  1.  1.  1. ]
 [0.7 0.7 0.7 0.  0.7 0.7 0.7 0.7 0.7 0.7]
 [1.  1.  1.  1.  0.  1.  1.  1.  1.  1. ]
 [1.  1. 

### Softmax loss

In [13]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = custom_layers.Softmax( 
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model = tf.keras.Model([x_input, y_input], logits)
model.compile(loss=None, optimizer='adam')

In [14]:
history_model_sm = []
for epoch in range(epochs):
    history_epoch = model_sm.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=0)
    history_model_sm.append(history_epoch)

In [17]:
T=20
softmax = lambda x: np.exp(x) #/np.sum(np.exp(x),1, keepdims=True)
yt_hat_std = np.array([softmax(model_sm.predict([x_test, y_test])) for _ in range(T)]) #np.argmax(y_test,1)
MC_pred_std = np.mean(yt_hat_std, 0)
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test,H_x_test_std,1)
loss_std = np.mean(utilities.loss_np(y_test,H_x_test_std,loss_mat))
print('Standard:\n')
print('Accuracy on optimal decision: %0.2f' % acc_std)
print('Expected loss: %0.2f' % loss_std)

ValueError: not enough values to unpack (expected 2, got 1)

### Loss calibrated softmax loss

In [ ]:
# For reproduceable results
importlib.reload(custom_layers)
importlib.reload(utilities)

# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')
h_input =  tf.keras.Input((1,), name = 'h_input')

logits = custom_layers.LCSoftmax( 
    10, 
    M,
    loss_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input, h_input])

model_sm_lc = tf.keras.Model([x_input, y_input, h_input], logits)
model_sm_lc.compile(loss=None, optimizer='adam')

In [ ]:
h_train = y_train
h_test = y_test
h_val = y_val

history_model_sm_lc = []
for epoch in range(epochs):
    history_epoch = model_sm_lc.fit([x_train, y_train, h_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1,
                 validation_data=([x_val, y_val, h_val], None))
#                  callbacks=[model_checkpoint])
    T = 10
    yt_hat = np.array([model_sm_lc.predict([x_train, y_train, h_train]) for _ in range(T)])
    h_train = utilities.optimal_h(yt_hat, loss_mat)
    history_model_sm_lc.append(history_epoch)
       

### One-vs-each loss

In [ ]:
# For reproduceable results
importlib.reload(custom_layers)
importlib.reload(utilities)
# np.random.seed(1337)

# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)


x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = custom_layers.OVELayer(
    5,
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_ove = tf.keras.Model([x_input, y_input], logits)
model_ove.compile(loss=None, optimizer='adam')

In [ ]:

history_model_ove = []
for epoch in range(epochs):
    history_epoch = model_ove.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1,
                 validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])

    history_model_ove.append(history_epoch)
       

In [ ]:
# For reproduceable results
importlib.reload(custom_layers)
importlib.reload(utilities)
# np.random.seed(1337)

# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)


x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = custom_layers.NCE(
    5,
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_nce = tf.keras.Model([x_input, y_input], logits)
model_nce.compile(loss=None, optimizer='adam')

In [ ]:

history_model_nce = []
for epoch in range(200):
    history_epoch = model_nce.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1,
                 validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])

    history_model_nce.append(history_epoch)
       

## Risk evaluation

In [ ]:
#  Softmax model

T = 10
yt_hat_std = np.array([model_sm.predict([x_test, y_test]) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)


# #  Softmax with loss calibration

# T = 10
# yt_hat_sm_lc = np.array([model_sm_lc.predict([x_test, y_test, h_test]) for _ in range(T)])
# H_x_test_sm_lc = utilities.optimal_h(yt_hat_sm_lc,loss_mat) 
# acc_sm_lc = accuracy_score(y_test, H_x_test_sm_lc)
# loss_sm_lc = np.mean(loss_mat[y_test, H_x_test_sm_lc])
# print('\nLoss calibrated softmax:\n')
# print('Accuracy on optimal decision: ', acc_sm_lc)
# print('Expected loss: ', loss_sm_lc)


# OVE model

T = 10 
yt_hat_ove = np.array([model_ove.predict([x_test, y_test]) for _ in range(T)])
H_x_test_ove = utilities.optimal_h(yt_hat_ove,loss_mat) 
acc_ove = accuracy_score(y_test, H_x_test_ove)
loss_ove = np.mean(loss_mat[y_test, H_x_test_ove])
print('\nOVE:\n')
print('Accuracy on optimal decision: ', acc_ove)
print('Expected loss: ', loss_ove)



# NCE model

T = 10 
yt_hat_nce = np.array([model_nce.predict([x_test, y_test]) for _ in range(T)])
H_x_test_nce = utilities.optimal_h(yt_hat_nce,loss_mat) 
acc_nce = accuracy_score(y_test, H_x_test_nce)
loss_nce = np.mean(loss_mat[y_test, H_x_test_nce])
print('\nNCE:\n')
print('Accuracy on optimal decision: ', acc_nce)
print('Expected loss: ', loss_nce)


In [ ]:
plt.rcParams['figure.figsize'] = (10, 10) # Width and height

classes = ['0','1','2','3','4','5','6','7','8','9']


cnf_matrix_std = confusion_matrix(y_test.astype('float'),np.round(H_x_test_std))
np.set_printoptions(precision=2)
cnf_matrix_sm_lc = confusion_matrix(y_test.astype('float'),np.round(H_x_test_sm_lc))
np.set_printoptions(precision=2)
cnf_matrix_ove = confusion_matrix(y_test.astype('float'),np.round(H_x_test_ove))
np.set_printoptions(precision=2)
cnf_matrix_nce = confusion_matrix(y_test.astype('float'),np.round(H_x_test_nce))
np.set_printoptions(precision=2)

plt.subplot(221)
utilities.plot_confusion_matrix(cnf_matrix_std, classes=classes, normalize=True,
                      title='Softmax')
plt.subplot(222)
utilities.plot_confusion_matrix(cnf_matrix_sm_lc, classes=classes, normalize=True,
                      title='LC Softmax')
plt.subplot(223)
utilities.plot_confusion_matrix(cnf_matrix_ove, classes=classes, normalize=True,
                      title='OVE')
plt.subplot(224)
utilities.plot_confusion_matrix(cnf_matrix_nce, classes=classes, normalize=True,
                      title='NCE')

In [ ]:
digits = [3,8]
np.set_printoptions(precision=3)
L = np.copy(I)
L[np.where(I==0)] = 1
L[[3,8]] = 0.7 # Select more important rows with lower loss in prediction
L[np.where(I==1)] = 0

loss_mat = L
M = 1.000

print("         TRUTH\n" )
string = 'PRED'
util_mat = M - loss_mat
for i in range(num_classes):
    if i < 4:
        print(string[i],'  ',str(i),': ',M - loss_mat[i])
    else:
        print('    ',str(i),': ',M - loss_mat[i])
        
# Set up weight for weighted cross entropy

class_weight = np.ones((num_classes))
class_weight[digits] = 1.4

print(loss_mat)

In [ ]:
import new_custom_layers
import custom_layers

### Stochastic loss calibration for softmax

In [ ]:
# For reproduceable results
importlib.reload(new_custom_layers)
importlib.reload(utilities)

# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

outputs = x_input
logits = new_custom_layers.DUCARLayer(
    5,
    num_classes, 
    util_mat,
    kernel_divergence_fn=scaled_kl)([outputs, y_input])

model_sm_slc = tf.keras.Model([x_input, y_input], logits)
model_sm_slc.compile(loss=None, optimizer='adam')

In [ ]:
# history_model_sm_slc = []
# for epoch in range(10):
history_epoch = model_sm_slc.fit([x_train, y_train],
             batch_size=batch_size,
             epochs=10,
             verbose=1)
#                  validation_data=([x_val, y_val, h_val], None))
#                  callbacks=[model_checkpoint])
#     T = 1
#     yt_hat = np.array([model_sm_slc.predict([x_train, y_train, h_train]) for _ in range(T)])
#     h_train = utilities.optimal_h(yt_hat, loss_mat)
#     history_model_sm_slc.append(history_epoch)
       
#     h_train = model_sm_slc.predict([x_train, y_train, h_train])

In [ ]:
#  Softmax with stochastic loss calibration
T = 10
yt_hat_sm_slc = np.array([np.exp(model_sm_slc.predict([x_test, y_test])) for _ in range(T)])
H_x_test_sm_slc = utilities.optimal_h(yt_hat_sm_slc,loss_mat) 
acc_sm_slc = accuracy_score(y_test, H_x_test_sm_slc)
loss_sm_slc = np.mean(loss_mat[y_test, H_x_test_sm_slc])
print('\nLoss calibrated softmax:\n')
print('Accuracy on optimal decision: ', acc_sm_slc)
print('Expected loss: ', loss_sm_slc)

In [ ]:
importlib.reload(new_custom_layers)
importlib.reload(utilities)


# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

outputs = x_input
# output = tfp.layers.DenseReparameterization(50,
#             activation = tf.nn.relu,
#             kernel_divergence_fn=scaled_kl)(x_input)
logits = custom_layers.DUCARLayer( 
    5,
    num_classes, 
    M,
    util_mat,
    kernel_divergence_fn=scaled_kl)([outputs, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [ ]:

history_model_sm = []
for epoch in range(10):
    history_epoch = model_sm.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1)
    history_model_sm.append(history_epoch)
       

In [ ]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

In [ ]:
loss_mat

In [ ]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

outputs = x_input
# output = tfp.layers.DenseReparameterization(50,
#             activation = tf.nn.relu,
#             kernel_divergence_fn=scaled_kl)(x_input)
logits = custom_layers.ARLayer(
    5,
    10,
    kernel_divergence_fn=scaled_kl)([outputs, y_input])

model_ar = tf.keras.Model([x_input, y_input], logits)
model_ar.compile(loss=None, optimizer='adam')

In [ ]:

history_model_ar = []
for epoch in range(10):
    history_epoch = model_ar.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1)
#                  validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
    history_model_ar.append(history_epoch)
       

In [ ]:
T = 10
yt_hat_ar = np.array([(model_ar.predict([x_test, y_test])) for _ in range(T)])
H_x_test_ar = utilities.optimal_h(yt_hat_ar,loss_mat) 
acc_ar = accuracy_score(y_test, H_x_test_ar)
loss_ar = np.mean(loss_mat[y_test, H_x_test_ar])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_ar)

In [ ]:
tf.RaggedTensor

In [ ]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.ARLayer( 
    5,
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [ ]:

history_model_sm = []
for epoch in range(10):
    history_epoch = model_sm.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1)
#                  validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
    history_model_sm.append(history_epoch)
       

In [ ]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

In [ ]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.Softmax( 
#     5,
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [ ]:

history_model_sm = []
for epoch in range(10):
    history_epoch = model_sm.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1)
#                  validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
    history_model_sm.append(history_epoch)
       

In [ ]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

In [ ]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(new_custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.OVELayer( 
    5,
    10, 
#     util_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [ ]:
history_epoch = model_sm.fit([x_train, y_train],
             batch_size=batch_size,
             epochs=10,
             verbose=1)
#                  validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
#     history_model_sm.append(history_epoch)
       

In [ ]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

In [ ]:
loss_mat

In [ ]:
loss_mat

In [ ]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(new_custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')
h_input =  tf.keras.Input((1,), name = 'h_input')

logits = custom_layers.SUCSoftmax( 
#     5,
    10, 
    util_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input, h_input])

model_sm = tf.keras.Model([x_input, y_input, h_input], logits)
model_sm.compile(loss=None, optimizer='adam')

T = 1
h_train = y_train[:]
yt_hat = np.array([model_sm.predict([x_train, y_train, h_train]) for _ in range(T)])
h_train = utilities.optimal_h(yt_hat, loss_mat)

In [ ]:
# history_model_sm_slc = []
for epoch in range(10):
    history_epoch = model_sm.fit([x_train, y_train, h_train],
             batch_size=batch_size,
             epochs=1,
             verbose=1)
    T = 1
    yt_hat = np.array([model_sm.predict([x_train, y_train, h_train]) for _ in range(T)])
    h_train = utilities.optimal_h(yt_hat, loss_mat)
#     h_train = model_sm_slc.predict([x_train, y_train, h_train])

In [ ]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

In [ ]:
rows = [[i] for i in range(num_classes)]
cols = [[i] for i in range(num_classes)]
vals = [[1] for i in range(num_classes)]

In [ ]:
util_mat = 1 - np.eye(num_classes)
loss_mat = np.eye(num_classes)

In [ ]:
util_mat

In [ ]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(new_custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.SUCSoftmax( 
#     5,
    num_classes, 
    rows, cols, vals,
#     util_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [ ]:
history_epoch = model_sm.fit([x_train, y_train],
         batch_size=batch_size,
         epochs=10,
         verbose=1)

In [ ]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

In [ ]:
model_sm.predict([x_test, y_test])

In [ ]:
rows

In [ ]:
cols

In [ ]:
importlib.reload(custom_layers)

In [ ]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = custom_layers.DUCSoftmax( 
#     5,
    num_classes, 
    util_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')